In [1]:
def calculate(label, is_multi, df):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    kosong = 0
    
    for index in range(0, len(df)):
        labels = df['label'][index].split('|')
        
        if type(df['pred_multiple'][index]) != float:
            preds = df['pred_multiple'][index].split('|')

            if label in labels and label in preds:
                tp += 1
            elif label not in labels and label not in preds:
                tn += 1
            elif label not in labels and label in preds:
                fp += 1
            elif label in labels and label not in preds:
                fn += 1
        elif label in labels:
            fn += 1
        else:
            tn += 1
    precision = round(tp / (tp + fp), 2)
    recall = round(tp / (tp + fn), 2)
    f1 = round(2 * ((precision * recall) / (precision + recall)), 2)
    accuracy = round((tp+tn)/(tp+tn+fp+fn), 2)
    print(tp, '\t', tn, '\t', fp, '\t', fn, '\t', '\t', precision, '\t', recall, '\t', f1, '\t', '\t', accuracy)
    return tp, tn, fp, fn, precision, recall, f1, accuracy

In [2]:
import pandas as pd

def run(tipe):
    df = pd.read_csv('Results/Categorization/'+ tipe +'.csv')
    sf = pd.DataFrame(columns=['TP', 'TN', 'FP', 'FN', 'precision', 'recall', 'f1', 'accuracy'])
    print('tp', '\t', 'tn', '\t', 'fp', '\t', 'fn', '\t', '\t', 'prec', '\t', 'rec', '\t', 'f1', '\t', '\t', 'accuracy')

    cat = ['AMBIENCE', 'FOOD', 'SERVICE', 'VALUE']
    for c in cat:
        a, b, c, d, e, f, g, h = calculate(c, True, df)
        sf = sf.append({'TP':a, 'TN':b, 'FP':c, 'FN':d, 'precision':e, 'recall':f, 'f1':g, 'accuracy': h}, ignore_index=True)
    sf.to_excel("Results/Calculation/"+ tipe +".xlsx")

In [5]:
run("lda")

tp 	 tn 	 fp 	 fn 	 	 prec 	 rec 	 f1 	 	 accuracy
109 	 482 	 11 	 97 	 	 0.91 	 0.53 	 0.67 	 	 0.85
363 	 214 	 73 	 49 	 	 0.83 	 0.88 	 0.85 	 	 0.83
140 	 476 	 52 	 31 	 	 0.73 	 0.82 	 0.77 	 	 0.88
124 	 408 	 149 	 18 	 	 0.45 	 0.87 	 0.59 	 	 0.76
